# Dermatologist AI mini project from Udacity
This mini project visually diagnoses melanoma, nevus and seborrheic keratosis, which is from [Udacity Deep Learning Nanodegree](https://www.udacity.com/course/deep-learning-nanodegree--nd101).<br>
This doesn't include data set for training because it doesn't belong to me, but the data and objective are pulled from the [2017 ISIC Challenge on Skin Lesion Analysis Towards Melanoma Detection](https://challenge.kitware.com/#challenge/583f126bcad3a51cc66c8d9a).<br>
https://isic-archive.com/#images <br>
(Because of 'Memory Error' issue, I delete python objects from time to time in this notebook)

## Import Datasets

In [1]:
from sklearn.datasets import load_files   
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(data_path, shuffle=None):
    kwargs = {}
    if shuffle != None:
        kwargs['shuffle'] = shuffle
    data = load_files(data_path, **kwargs)
    img_files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    return img_files, targets

train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test', shuffle=False)

# load lables
label_name = [item[11:-1] for item in sorted(glob("data/train/*/"))]

D:\Users\Wonjoon\Miniconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print('train_files size: {}'.format(len(train_files)))
print('train_files shape: {}'.format(train_files.shape))
print('target shape: {}'.format(train_targets.shape))
print(label_name)

train_files size: 2000
train_files shape: (2000,)
target shape: (2000, 3)
['melanoma', 'nevus', 'seborrheic_keratosis']


In [3]:
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from tqdm import tqdm


def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(384, 256))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(image_paths):
    return np.vstack([path_to_tensor(path) for path in image_paths])


## Network with transfer learning
### Loading images into tensors

In [4]:
train_tensors = paths_to_tensor(tqdm(train_files))
valid_tensors = paths_to_tensor(tqdm(valid_files))
test_tensors = paths_to_tensor(tqdm(test_files))

print(train_tensors.shape)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [03:17<00:00,  3.04it/s]


(2000, 384, 256, 3)


### Image augmentation
To enlarge train data I tried image augmentation using flip.
At first I tried to transform using flip, zoom and shift, but I thought it can twist training features much because medical images are very sensitive, so I tried to use flip only.
However __image transform didn't help this network and dataset, so I disabled it in this project__.

In [5]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator


apply_train_image_transform = False

if apply_train_image_transform:
    # Caution: Doesn't guarantee prevention of duplication.
    datagen_train = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True)
    
    datagen_train.fit(train_tensors)
    shape = (train_tensors.shape[0] * 2,) + train_tensors.shape[1:]
    generated = np.ndarray(shape=shape)
    for i, image in tqdm(enumerate(train_tensors)):
        generated[i] = datagen_train.random_transform(image)
    
    train_tensors = np.concatenate((train_tensors, generated))
    train_targets = train_targets.repeat(2, axis=0)

### Transfer learning using Inception Resnet V2

In [6]:
# train_imgs_preprocess is created below
train_imgs_preprocess = preprocess_input(train_tensors)
valid_imgs_preprocess = preprocess_input(valid_tensors)
test_imgs_preprocess = preprocess_input(test_tensors)
del train_tensors, valid_tensors, test_tensors

In [7]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
transfer_model = InceptionResNetV2(include_top=False)

train_data = transfer_model.predict(train_imgs_preprocess)
valid_data = transfer_model.predict(valid_imgs_preprocess)
test_data = transfer_model.predict(test_imgs_preprocess)

del train_imgs_preprocess, valid_imgs_preprocess, test_imgs_preprocess
print(train_data.shape)

(2000, 10, 6, 1536)


In [8]:
import gc
gc.collect()

358680

### Network Model

In [9]:
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

my_model = Sequential()

my_model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
my_model.add(Dropout(0.2))
my_model.add(Dense(1024, activation='relu'))
my_model.add(Dropout(0.2))

my_model.add(Dense(3, activation='softmax'))

my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1573888   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 1,576,963
Trainable params: 1,576,963
Non-trainable params: 0
_________________________________________________________________


In [10]:
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
from keras.callbacks import ModelCheckpoint


checkpoint_filepath = 'saved_models/weights.best.my.hdf5'

my_checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1, save_best_only=True)

my_model.fit(train_data, train_targets, 
          validation_data=(valid_data, valid_targets),
          epochs=60, batch_size=200, callbacks=[my_checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/60
2000/2000 [==============================] - ETA: 15s - loss: 1.3028 - acc: 0.27 - ETA: 7s - loss: 2.0917 - acc: 0.4900 - ETA: 2s - loss: 2.3659 - acc: 0.590 - ETA: 1s - loss: 2.2341 - acc: 0.548 - ETA: 0s - loss: 1.9577 - acc: 0.526 - ETA: 0s - loss: 1.7383 - acc: 0.560 - 2s 1ms/step - loss: 1.6664 - acc: 0.5650 - val_loss: 1.2237 - val_acc: 0.5600

Epoch 00001: val_loss improved from inf to 1.22366, saving model to saved_models/weights.best.my.hdf5
Epoch 2/60
2000/2000 [==============================] - ETA: 0s - loss: 0.8903 - acc: 0.755 - ETA: 0s - loss: 0.9057 - acc: 0.718 - ETA: 0s - loss: 0.8479 - acc: 0.715 - ETA: 0s - loss: 0.8314 - acc: 0.706 - ETA: 0s - loss: 0.8391 - acc: 0.689 - 1s 251us/step - loss: 0.8339 - acc: 0.6870 - val_loss: 0.9172 - val_acc: 0.5733

Epoch 00002: val_loss improved from 1.22366 to 0.91720, saving model to saved_models/weights.best.my.hdf5
Epoch 3/60
2000/2000 [==============================] 

Epoch 00020: val_loss did not improve from 0.71352
Epoch 21/60
2000/2000 [==============================] - ETA: 0s - loss: 0.5034 - acc: 0.840 - ETA: 0s - loss: 0.5455 - acc: 0.790 - ETA: 0s - loss: 0.5278 - acc: 0.790 - ETA: 0s - loss: 0.5411 - acc: 0.782 - ETA: 0s - loss: 0.5332 - acc: 0.786 - 1s 251us/step - loss: 0.5271 - acc: 0.7930 - val_loss: 0.7286 - val_acc: 0.7133

Epoch 00021: val_loss did not improve from 0.71352
Epoch 22/60
2000/2000 [==============================] - ETA: 0s - loss: 0.5044 - acc: 0.810 - ETA: 0s - loss: 0.5315 - acc: 0.785 - ETA: 0s - loss: 0.5133 - acc: 0.795 - ETA: 0s - loss: 0.5176 - acc: 0.794 - ETA: 0s - loss: 0.5102 - acc: 0.799 - 1s 253us/step - loss: 0.5130 - acc: 0.7960 - val_loss: 0.7242 - val_acc: 0.7000

Epoch 00022: val_loss did not improve from 0.71352
Epoch 23/60
2000/2000 [==============================] - ETA: 0s - loss: 0.4919 - acc: 0.805 - ETA: 0s - loss: 0.5097 - acc: 0.801 - ETA: 0s - loss: 0.5098 - acc: 0.804 - ETA: 0s - loss: 0.50

2000/2000 [==============================] - ETA: 0s - loss: 0.4436 - acc: 0.840 - ETA: 0s - loss: 0.3948 - acc: 0.848 - ETA: 0s - loss: 0.3960 - acc: 0.841 - ETA: 0s - loss: 0.4099 - acc: 0.837 - ETA: 0s - loss: 0.4187 - acc: 0.833 - 1s 252us/step - loss: 0.4155 - acc: 0.8355 - val_loss: 0.8093 - val_acc: 0.6733

Epoch 00041: val_loss did not improve from 0.66539
Epoch 42/60
2000/2000 [==============================] - ETA: 0s - loss: 0.5475 - acc: 0.760 - ETA: 0s - loss: 0.4421 - acc: 0.810 - ETA: 0s - loss: 0.4197 - acc: 0.832 - ETA: 0s - loss: 0.4104 - acc: 0.835 - ETA: 0s - loss: 0.4037 - acc: 0.833 - 0s 250us/step - loss: 0.4029 - acc: 0.8345 - val_loss: 0.6832 - val_acc: 0.7267

Epoch 00042: val_loss did not improve from 0.66539
Epoch 43/60
2000/2000 [==============================] - ETA: 0s - loss: 0.3193 - acc: 0.890 - ETA: 0s - loss: 0.3611 - acc: 0.860 - ETA: 0s - loss: 0.3858 - acc: 0.843 - ETA: 0s - loss: 0.4060 - acc: 0.832 - ETA: 0s - loss: 0.4118 - acc: 0.833 - 1s 254u

In [12]:
my_model.load_weights(checkpoint_filepath)

### Evaluation

In [13]:
import csv


my_predictions = [my_model.predict(np.expand_dims(feature, axis=0)) for feature in test_data]

# test_accuracy = 100 * np.sum(np.array(my_predictions)==np.argmax(test_targets, axis=1)) / len(my_predictions)
# print('Test accuracy: %.4f%%' % test_accuracy)

with open('my_transfer.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])

In [ ]:
Result of ROC AUC
Category 1 Score: 0.759
Category 2 Score: 0.877
Category 3 Score: 0.818

### Cleaning up python object

In [15]:
del train_data, valid_data, test_data, transfer_model, my_model
gc.collect()

1177

## Network without transfer learning
### Loading images into tensors

In [16]:
train_tensors = paths_to_tensor(tqdm(train_files))
train_tensors = train_tensors.astype('float32') / 255

valid_tensors = paths_to_tensor(tqdm(valid_files))
valid_tensors = valid_tensors.astype('float32') / 255

test_tensors = paths_to_tensor(tqdm(test_files))
test_tensors = test_tensors.astype('float32') / 255

print(train_tensors.shape)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [03:22<00:00,  2.97it/s]


(2000, 384, 256, 3)


## Network Model 2 for CNN without transfer learning

In [26]:
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

my_model = Sequential()

my_model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', 
                        input_shape=train_tensors.shape[1:]))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.2))

my_model.add(Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.1))

my_model.add(Conv2D(filters=2048, kernel_size=3, padding='same', activation='relu'))
my_model.add(MaxPooling2D(pool_size=2))
my_model.add(Dropout(0.1))

# my_model.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
# my_model.add(MaxPooling2D(pool_size=2))
# my_model.add(Dropout(0.1))

# my_model.add(Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'))
# my_model.add(MaxPooling2D(pool_size=2))
# my_model.add(Dropout(0.1))

my_model.add(GlobalAveragePooling2D())

my_model.add(Dense(3, activation='softmax'))

my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_211 (Conv2D)          (None, 384, 256, 16)      448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 192, 128, 16)      0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 192, 128, 16)      0         
_________________________________________________________________
conv2d_212 (Conv2D)          (None, 192, 128, 64)      9280      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 96, 64, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 96, 64, 64)        0         
_________________________________________________________________
conv2d_213 (Conv2D)          (None, 96, 64, 256)       147712    
__________

In [27]:
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
gc.collect()

11130

In [30]:
from keras.callbacks import ModelCheckpoint
import os


checkpoint_filepath = 'saved_models/weights.best.my.hdf5'

my_checkpointer = ModelCheckpoint(filepath=checkpoint_filepath,
                               verbose=1, save_best_only=True)

my_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=40, batch_size=100, callbacks=[my_checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/40
2000/2000 [==============================] - ETA: 1:11 - loss: 0.7374 - acc: 0.690 - ETA: 38s - loss: 0.7194 - acc: 0.705 - ETA: 27s - loss: 0.7487 - acc: 0.68 - ETA: 21s - loss: 0.7840 - acc: 0.66 - ETA: 17s - loss: 0.7776 - acc: 0.67 - ETA: 14s - loss: 0.7851 - acc: 0.66 - ETA: 12s - loss: 0.7837 - acc: 0.66 - ETA: 11s - loss: 0.7814 - acc: 0.66 - ETA: 9s - loss: 0.7904 - acc: 0.6533 - ETA: 8s - loss: 0.7887 - acc: 0.657 - ETA: 7s - loss: 0.7787 - acc: 0.666 - ETA: 6s - loss: 0.7772 - acc: 0.667 - ETA: 5s - loss: 0.7693 - acc: 0.674 - ETA: 4s - loss: 0.7741 - acc: 0.671 - ETA: 3s - loss: 0.7657 - acc: 0.677 - ETA: 2s - loss: 0.7688 - acc: 0.675 - ETA: 2s - loss: 0.7597 - acc: 0.680 - ETA: 1s - loss: 0.7515 - acc: 0.686 - ETA: 0s - loss: 0.7477 - acc: 0.688 - 15s 7ms/step - loss: 0.7509 - acc: 0.6890 - val_loss: 0.8336 - val_acc: 0.5400

Epoch 00001: val_loss improved from inf to 0.83356, saving model to saved_models/weights.be

2000/2000 [==============================] - ETA: 9s - loss: 0.6723 - acc: 0.730 - ETA: 9s - loss: 0.7606 - acc: 0.700 - ETA: 8s - loss: 0.7420 - acc: 0.713 - ETA: 8s - loss: 0.7617 - acc: 0.690 - ETA: 7s - loss: 0.7426 - acc: 0.692 - ETA: 7s - loss: 0.7379 - acc: 0.693 - ETA: 6s - loss: 0.7417 - acc: 0.688 - ETA: 6s - loss: 0.7469 - acc: 0.682 - ETA: 5s - loss: 0.7431 - acc: 0.683 - ETA: 5s - loss: 0.7332 - acc: 0.689 - ETA: 4s - loss: 0.7339 - acc: 0.685 - ETA: 4s - loss: 0.7392 - acc: 0.681 - ETA: 3s - loss: 0.7361 - acc: 0.683 - ETA: 3s - loss: 0.7441 - acc: 0.681 - ETA: 2s - loss: 0.7361 - acc: 0.686 - ETA: 2s - loss: 0.7325 - acc: 0.687 - ETA: 1s - loss: 0.7320 - acc: 0.688 - ETA: 1s - loss: 0.7274 - acc: 0.691 - ETA: 0s - loss: 0.7293 - acc: 0.690 - 11s 5ms/step - loss: 0.7294 - acc: 0.6895 - val_loss: 0.7839 - val_acc: 0.5267

Epoch 00009: val_loss improved from 0.78957 to 0.78387, saving model to saved_models/weights.best.my.hdf5
Epoch 10/40
2000/2000 [========================

2000/2000 [==============================] - ETA: 9s - loss: 0.6879 - acc: 0.720 - ETA: 9s - loss: 0.7317 - acc: 0.685 - ETA: 8s - loss: 0.7430 - acc: 0.676 - ETA: 8s - loss: 0.7245 - acc: 0.690 - ETA: 7s - loss: 0.7152 - acc: 0.688 - ETA: 7s - loss: 0.7059 - acc: 0.691 - ETA: 6s - loss: 0.7101 - acc: 0.687 - ETA: 6s - loss: 0.7022 - acc: 0.695 - ETA: 5s - loss: 0.7112 - acc: 0.687 - ETA: 5s - loss: 0.7011 - acc: 0.697 - ETA: 4s - loss: 0.6953 - acc: 0.703 - ETA: 4s - loss: 0.6942 - acc: 0.703 - ETA: 3s - loss: 0.6857 - acc: 0.706 - ETA: 3s - loss: 0.6765 - acc: 0.708 - ETA: 2s - loss: 0.6696 - acc: 0.710 - ETA: 2s - loss: 0.6651 - acc: 0.713 - ETA: 1s - loss: 0.6852 - acc: 0.710 - ETA: 1s - loss: 0.6850 - acc: 0.713 - ETA: 0s - loss: 0.6811 - acc: 0.714 - 11s 5ms/step - loss: 0.6835 - acc: 0.7120 - val_loss: 0.8185 - val_acc: 0.6467

Epoch 00017: val_loss did not improve from 0.76749
Epoch 18/40
2000/2000 [==============================] - ETA: 9s - loss: 0.7741 - acc: 0.650 - ETA: 9s

Epoch 00025: val_loss did not improve from 0.76621
Epoch 26/40
2000/2000 [==============================] - ETA: 9s - loss: 0.5028 - acc: 0.840 - ETA: 9s - loss: 0.5398 - acc: 0.790 - ETA: 8s - loss: 0.5654 - acc: 0.776 - ETA: 8s - loss: 0.6085 - acc: 0.747 - ETA: 7s - loss: 0.6146 - acc: 0.748 - ETA: 7s - loss: 0.6173 - acc: 0.748 - ETA: 6s - loss: 0.6212 - acc: 0.741 - ETA: 6s - loss: 0.6184 - acc: 0.743 - ETA: 5s - loss: 0.6130 - acc: 0.748 - ETA: 5s - loss: 0.6188 - acc: 0.747 - ETA: 4s - loss: 0.6207 - acc: 0.746 - ETA: 4s - loss: 0.6217 - acc: 0.742 - ETA: 3s - loss: 0.6160 - acc: 0.743 - ETA: 3s - loss: 0.6199 - acc: 0.741 - ETA: 2s - loss: 0.6151 - acc: 0.744 - ETA: 2s - loss: 0.6104 - acc: 0.746 - ETA: 1s - loss: 0.6147 - acc: 0.743 - ETA: 1s - loss: 0.6139 - acc: 0.743 - ETA: 0s - loss: 0.6209 - acc: 0.740 - 11s 5ms/step - loss: 0.6189 - acc: 0.7370 - val_loss: 0.7775 - val_acc: 0.6600

Epoch 00026: val_loss did not improve from 0.76621
Epoch 27/40
2000/2000 [================


Epoch 00034: val_loss did not improve from 0.73744
Epoch 35/40
2000/2000 [==============================] - ETA: 9s - loss: 0.6003 - acc: 0.740 - ETA: 9s - loss: 0.6006 - acc: 0.740 - ETA: 8s - loss: 0.5670 - acc: 0.760 - ETA: 8s - loss: 0.5525 - acc: 0.772 - ETA: 7s - loss: 0.5414 - acc: 0.782 - ETA: 7s - loss: 0.5291 - acc: 0.785 - ETA: 6s - loss: 0.5327 - acc: 0.780 - ETA: 6s - loss: 0.5562 - acc: 0.765 - ETA: 5s - loss: 0.5845 - acc: 0.763 - ETA: 5s - loss: 0.5719 - acc: 0.766 - ETA: 4s - loss: 0.5725 - acc: 0.764 - ETA: 4s - loss: 0.5655 - acc: 0.765 - ETA: 3s - loss: 0.5706 - acc: 0.766 - ETA: 3s - loss: 0.5811 - acc: 0.760 - ETA: 2s - loss: 0.5803 - acc: 0.762 - ETA: 2s - loss: 0.5841 - acc: 0.760 - ETA: 1s - loss: 0.5799 - acc: 0.761 - ETA: 1s - loss: 0.5711 - acc: 0.769 - ETA: 0s - loss: 0.5706 - acc: 0.769 - 11s 5ms/step - loss: 0.5727 - acc: 0.7675 - val_loss: 0.8587 - val_acc: 0.6533

Epoch 00035: val_loss did not improve from 0.73744
Epoch 36/40
2000/2000 [===============

In [31]:
my_model.load_weights(checkpoint_filepath)

In [32]:
import csv


my_predictions = [my_model.predict(np.expand_dims(feature, axis=0)) for feature in test_tensors]

# test_accuracy = 100 * np.sum(np.array(my_predictions)==np.argmax(test_targets, axis=1)) / len(my_predictions)
# print('Test accuracy: %.4f%%' % test_accuracy)

with open('my_cnn.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])

In [33]:
my_predictions

[array([[0.0537998 , 0.3720544 , 0.57414573]], dtype=float32),
 array([[0.23505117, 0.40950906, 0.35543972]], dtype=float32),
 array([[0.25806636, 0.6062299 , 0.13570368]], dtype=float32),
 array([[0.4129069 , 0.5557064 , 0.03138682]], dtype=float32),
 array([[0.23806821, 0.68993616, 0.07199564]], dtype=float32),
 array([[0.14545488, 0.8195845 , 0.03496062]], dtype=float32),
 array([[0.17491573, 0.72190344, 0.10318081]], dtype=float32),
 array([[0.15154465, 0.5129464 , 0.335509  ]], dtype=float32),
 array([[0.2722508 , 0.68131775, 0.0464314 ]], dtype=float32),
 array([[0.4218841 , 0.53918225, 0.03893368]], dtype=float32),
 array([[0.24519944, 0.72761863, 0.02718199]], dtype=float32),
 array([[0.21201548, 0.51336855, 0.27461597]], dtype=float32),
 array([[0.18136051, 0.4665843 , 0.3520552 ]], dtype=float32),
 array([[0.16907156, 0.34839672, 0.4825317 ]], dtype=float32),
 array([[0.0588286 , 0.9238437 , 0.01732777]], dtype=float32),
 array([[0.2415644 , 0.7562038 , 0.00223186]], dtype=fl

In [ ]:
Result of ROC AUC
Category 1 Score: 0.575
Category 2 Score: 0.849
Category 3 Score: 0.712